# Implementation

In the previous code, I have implemented all the tools needed to make a fully automised monthly report maker (except for the mailing part). There are several main goals to achieve with this function:

1. Data treatment (NaNs esp)
2. Creation of relevant numbers and graphs
3. Creation of word document
4. Saving word document report and graphs in a dedicated folder

For every CPO that needs a report.

### CPO reports

There are 2 types of CPOs: EZ drive and Non-EZ drive

All reports need:

**Numbers**
-Nombre de points de charges
-Nombres de recharges mensuel
-Temps moyen d'une charge (min)
-Consommation Mensuelle (kWh)

**Graphs**
-Daily number of charges
-Daily amount of charges

**Non EZ-drive**
Estimation cout mensuel

**EZ-drive**
Retribution sur l'energie consommee




## 1. Read and treat database

In [83]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
import os
import datetime
import time
from docx.shared import Inches


In [2]:
#enter dates of report
month = '08'
year = '2022'
date_start = '01'
date_end = '31'

charge_events_filename = 'cpochargeevents_'+date_start+'.'+month+'.'+year+' - '+date_end+'.'+month+'.'+year+'.csv'
charge_events = pd.read_csv(charge_events_filename,skiprows=1) #load the csv charge events file into pandas dataframe
charge_events['Energy (Wh)'] = charge_events['Energy (Wh)']/1000 #change Wh to kWh
charge_events.rename(columns = {'Energy (Wh)':'Energy (kWh)'})
time_24 = ['Created','Stop time','Start time']
charge_events[time_24] = charge_events[time_24].apply(pd.to_datetime,format = '%Y-%m-%d %H:%M:%S') #apply date time format to columns w dates
missing_stop_time_idx = np.where(charge_events['Stop time'].isnull())
charge_events['Stop time'].loc[missing_stop_time_idx] = charge_events['Start time'].loc[missing_stop_time_idx]+timedelta(hours=1) #adding 1 hour to NaN values
charge_events['Date'] = charge_events['Start time'].dt.date #create a new column with just the date of the start of the charge




/home/stella/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## 2. Groupby - for every CPO, generate the necessary data for the report

Loop for every unique CPO name to create and save the two graphs and report.


In [81]:
monthly_filepath = '/home/stella/Documents/GT/monthly_reports/'
try:
    os.mkdir(monthly_filepath+month+'-'+year)#creates a folder for current month
except FileExistsError: # in case the folder alrdy exists
    pass

try:
    os.mkdir(monthly_filepath+month+'-'+year+'/E.Zdrive') #creates folder for ez drive n non ezdrive 
except FileExistsError:
    pass

try:
        os.mkdir(monthly_filepath+month+'-'+year+'/Non E.Zdrive')
except FileExistsError:
    pass

# in the case of non-EZ drive, this is the CPO. If EZ drive, this is the name of the station

for CPO in charge_events['CPO name'].unique():
    
    #if the CPO is EZ drive, make a report for each station
    if CPO != 'EZ DRIVE':

        try:
            os.mkdir(monthly_filepath+month+'-'+year+'/Non E.Zdrive/'+CPO)#creates a folder for current month
        except FileExistsError: # in case the folder alrdy exists
            pass  
    
            
        test_CPO = charge_events.groupby(['CPO name']).get_group(CPO) #get the info for a single CPO
        test_CPO_energy = test_CPO.groupby('Date')['Energy (Wh)'].sum()#sum all the energy used by this cpo for each date he/she recharged 
        test_CPO_count = test_CPO.groupby('Date')['Charge ID'].count()#count each time they used a borne for every date

        #plot daily energy bar chart
        plt.figure()
        bar_energy = test_CPO_energy.plot(kind= 'bar',figsize=(8,3))
        bar_energy.set_ylabel('Energy (Wh)')
        bar_energy.set_title('Energie consommée par '+ CPO)
        bar_energy.set_xticklabels(pd.to_datetime(test_CPO_energy.index).date,rotation = 45)
        bar_energy.figure.savefig(monthly_filepath+month+'-'+year+'/Non E.Zdrive/'+CPO+'/bar_energy.png',dpi=300,bbox_inches='tight')
        plt.close()

        #plot daily usage count bar chart
        plt.figure()
        bar_count = test_CPO_count.plot(kind='bar',figsize=(8,3))
        bar_count.set_ylabel('Nombre de recharges')
        bar_count.set_title('Nombre de recharges par '+ CPO)
        bar_count.set_xticklabels(pd.to_datetime(test_CPO_energy.index).date,rotation = 45)
        bar_count.set_yticks(test_CPO_count.unique())
        bar_count.figure.savefig(monthly_filepath+month+'-'+year+'/Non E.Zdrive/'+CPO+'/bar_count.png',dpi=300,bbox_inches='tight')
        plt.close()

        #get other relevant information needed for the report
        retribution = test_CPO['Price'].sum() #check w clemence this is how its calculated
        no_de_points = len(test_CPO['Station ID'].unique())
        no_de_recharges_mensuels = test_CPO_count.sum()
        temps_moyen_de_charge = test_CPO['Duration'].mean()
        conso_total = test_CPO_energy.sum()

        
        




In [23]:
charge_events_EZ =  charge_events[charge_events['CPO name']=='EZ DRIVE']
EZ_stations = charge_events_EZ['Station name'].unique()

In [55]:
a = test_CPO['Location type'].unique()[0]


### Creating a Document 


In [93]:
#def create_report(CPO,month,year,test_CPO):
document = Document()

header = document.sections[0].header
paragraph = header.paragraphs[0]

logo_run = paragraph.add_run()
logo_run.add_picture("logo.jpg", width=Inches(1))

text_run.text = '\t' + "Rapport de télégestion IRVE" # For center align of text
text_run.style = "Heading 1 Char"

document.add_heading("Rapport de télégestion IRVE", 0) #titre
document.add_heading(CPO + ' | '+month+' / '+ year,1) #sous-titre
if len(test_CPO['Location type'].unique()) == 1: #publique ou privee, departement
    document.add_heading(test_CPO['Location type'].unique(),4) 
else: 
    document.add_heading(' & '.join(test_CPO['Location type'].unique()),4) 

document.add_heading('Rapport Opérationel',2)
para = document.add_paragraph('Nombre de points de charge = '+str(no_de_points)+'\nNombre de recharges mensuel = '+str(no_de_recharges_mensuels)+"\nTemps moyen d'une charge (min) = "+  str(int(temps_moyen_de_charge))+'\nConsommation mensuelle (kWh) = '+str(np.round(conso_total,2)))
document.add_paragraph("_____________________________________________")
document.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
document.add_heading('Nombre de recharges',2)
bar_count_png = monthly_filepath+month+'-'+year+'/Non E.Zdrive/'+CPO+'/bar_count.png'
document.add_picture(bar_count_png,width=Inches(3.5))
document.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
document.add_paragraph("_____________________________________________")
document.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
document.add_heading('Energie consommée en kWh',2)
bar_energy_png = monthly_filepath+month+'-'+year+'/Non E.Zdrive/'+CPO+'/bar_energy.png'
document.add_picture(bar_energy_png,width=Inches(3.5))
document.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER


document.save('test_report.docx')
##later add a  country thingy


